<a href="https://colab.research.google.com/github/Sakha-Language-Processing/wordforms/blob/main/verbs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Автоматизированный поиск и стемминг глагольных словоформ по длинным окончаниям

Из корпуса wordforms.csv (см. https://www.kaggle.com/c/sakha-lemmatizer), состоящего из 98568 словоформ с подсчетом частоты. 

Выявлено 4622 словоформ и 1483 глаголов. В первоначальном тексте было 23294 вхождений данных 4622 словоформ глаголов.

В первоначальном тексте из 1028838 слов было не менее 23294 вхождений (нужно умножать в разы).

## Правила
Мы используем базу данных affixes(type:list) из 194 глагольных окончаний для выявления глаголов по двум правилам:
1. Ищем окончание в слове и обрезаем слово, начиная с первой буквы окончания.
2. С окончаниями работаем после замены гласных по базе данных dvowels(type:dictionary).

## Исключения
Такой подход имеет исключения. Приведем пример: радиоҕа --правило2--> радыаҕа
--правило1--> рад.
С целью избежать  ошибки на таких исключениях 
1. Гласных 'ио', 'иу', 'иа' заменяются на 'юю'.
2. Редкие исключения прописаны в базе данных exceptions(type:list).

Исключения могут дополняться.

## Ограничения области применения
Подход в принципе не может выявить глаголы, которые в корпусе встречаются только с короткими окончаниями. 

## Аналогичные подходы
Подход с тремя уникальными окончаниями был продемонстрирован в программе verbs_aacchy.


In [63]:
# Окончания к правилу 1
affixes = ['ыһыннарыһыннарара', 'ыһыннарара',
 'ыһыннардахха', 'ыһыннарыһыннарары', 'батаххыта',
 'ымаарыҥытый', 'ыһыннарарга', 'ыһыннарарыгар',
 'ыннарбатах',
 'батаххытыта', 'батахтаахтара',
 'ыспатаҕына', 'ыһыахтаахпытын', 'быттаахтара',
 'ыһыннарарын', 'лыахтаахтар','лархытыта',
 'батарбыт', 'ымаайаллар', 'быттаахпыт',
 'ыахтаахпыт', 'ымаарыҥытытый', 'быттаахтар',
 'ыһыннарар', 'ыаҕай', 'ыһыахпытын', 'ыһыннартаата',
 'батаххыт', 'ымаарыҥытытыый', 'ыһыннарда',
 'батахтааҕа', 'ыһыннараҕын', 'батахтааххыт',
 'ааччыта', 'ымыыһылар', 'ыһыннарбыттарыгар',
 'ыахпытытый', 'ыһыннарыыга', 'ааччылара',
 'батахтаахпыт', 'ыһыннаралларыгар', 'батахтарыта',
 'ыһыннарыһыннарыахпын', 'ыспатаҕын', 'батахпыта',
 'ымаарыҥ', 'батахтаахтар', 'ыахтааххыт',
 'ыһыахтааҕый', 'ымыахтарай', 'ыһыннарыахтааххын',
 'ааччыбыт', 'таныллыбыт', 'ыһыннарыаҕыҥ',
 'батахтааххын', 'ааччылар', 'батахпытыта',
 'ыһыаххайыҥ', 'ымыахпытый', 'ыллыаҕа',
 'ымаайаҕыт', 'ымаарыый', 'ыаҕаҥ',
 'ыаҕам', 'ыахтарай', 'ыаххытый',
 'лылар', 'ыспатаххына', 'ыһыннарыылара',
 'ымыыһыгыт', 'ыспаттык', 'ыһыаҕыҥ', 'ыстахтара',
 'батаҕыҥ', 'ыаҕа', 'батахпыт', 'батыбыт',
 'баппыт', 'ыһыннардылар', 'быттааҕым',
 'батылар', 'ыһыннарбат', 'ыахпытый',
 'ааччыгыт', 'ыстахпытына', 'ыһарбар',
 'ыһалларынан', 'ымаайабыт', 'быккын',
 'ыыһылар', 'ыахтара', 'бытыҥ', 'лаллар',
 'ымыаххытыный', 'ааччыҥ', 'ыстым', 'ыһалларый',
 'лаҕыта', 'батахтара', 'ыстаҕына', 'батахтаах',
 'батым', 'ымаҥ', 'быттаахпын', 'ыаххыт', 'лархыта',
 'ыллыбыт', 'лаҕым', 'лаҕыҥ', 'бытым', 'быттааххыт',
 'баттара', 'быппын', 'аарыҥытый', 'лыбыт',
 'ыспатылар', 'ыһыахтарын', 'аарыҥытыый',
 'аайаҕыт', 'ымаарый', 'ыахтаах', 'лархыт',
 'ыһыахпын', 'ыһыннарбытын', 'аарый',
 'ыһыах', 'ыһыннарыахтарын', 'аарыый',
 'ыспатахтара', 'батаҕым', 'ыһыннаран',
 'батахтааҕыҥ', 'ымаарай', 'аайаллар',
 'ыһыннарбытыгар', 'ыһыннарыыттан',
 'ааччым', 'ыспат', 'ларпыта', 'баттар',
 'ааччы', 'ыһыннарыы', 'ыстахтарына',
 'ыахпыт', 'ыһыннараары', 'батыҥ',
 'аарыҥ', 'ыстыгыт', 'лартарыта',
 'ыспаккын', 'ыһыннарыа', 'ыһаргытыгар',
 'ыһыннарбыта', 'ыспаппыт', 'ыстыннар',
 'батахтааҕым', 'ыһыннар', 'ымыыһыбыт',
 'ларпытыта', 'быттаах', 'ыстабы',
 'лыгыт', 'ымыым', 'быттааҕыҥ',
 'ыһарбытыгар', 'ыспыппытын',
 'ыспыппыттан', 'ымаар', 'батах', 'ыыһыгыт',
 'быттааҕа', 'лартара', 'батаргыт',
 'баккыт', 'ыстаххына', 'ыспытай',
 'ыспыттыы', 'батаҕыта', 'аайабыт',
 'ыһыай', 'ыстала', 'ыстахпыт',
 'ыһынным', 'ыахайыҥ', 'ларпыт',
 'ыспыккыный', 'ыстахтарын', 'ымыаҕыҥ',
 'батар', 'батаҕа', 'батыгыт', 'ыахтар',
 'ыыһыбыт', 'ыаххыный']
# В правиле 1 возможны только короткие окончания
affixes = [item for item in affixes if len(item)>3]

# Правила замены гласных 2
dvowels = {'у':'ы','е':'е','и':'ы','ү':'ы','о':'а','э':'а',
              'я':'я','ю':'ю','ө':'а','ё':'е','а':'а'}

# Исключения 1 -- фонетические на заимствованных словах
exceptions_loaned = {'ио':'юю', 'иа':'юю', 'иу':'юю'}

# Исключения 2 -- редкие 
exceptions = [  # -батах 
'аббат','набат','рабат','шабат',
'вомбат','изобат','комбат','ниобат','санбат','сан',
'акробат','акрбо','целибат','адсорбат','стилобат','стройбат','строй',
'штрафбат','медсанбат','медсан','стереобат','гидроизобат',
'изотермобат','теледебат','теледе',
                # -ыстым
'недопустимо', 'недоп', 'допустимо', 'доп',
                # -ыспат
'сулууспа','сулу','ветсулууспа','ветсулу', 'ветсулууспа', 'спецсулууспа', 
'метеосулууспа', 'радиосулууспа', 'разведсулууспа', 'авиаметеосулууспа', 
'гидрометеосулууспа', 'гидрометеосулу', 
                # -ааччы
'бэдэрээччит','бэдэр',
        # -ардаахпыт быраҕылынна фитобаардаахпыт эбэтэр фитобардаахпыт   
]



import pandas as pd
import numpy as np
from collections import Counter

# Функция. Правила 1: выявление окончаний 
def isitaverb(x):
    global affixes
    for item in affixes:
       if item in x and x.index(item)>2:
           return x.index(item)
    return 0


# Функция. Правила 2: замена гласных 
def change_to_a(x):
    for loaned, ersatz in exceptions_loaned.items():
        x = x.replace(loaned, ersatz)   
    return ''.join([dvowels[char] if char in dvowels.keys() else char for char in x])

# Функция. Упорядочивание окончаний для Правила 1.
# Пример: окончание 'ыныах' ищем до поиска 'ыах' 
def sort_inclusion_one_round(x):
    new = []
    for item in x:
       memorize = new
       must_be_inserted_to_head = False
       for item2 in memorize: 
          if item2 in item: # True = добавляем подстроку, которая встречалась
              must_be_inserted_to_head = True
       if must_be_inserted_to_head:       
              new.insert(0,item) 
       else:
              new.append(item) 
    return new

# Функция. Упорядочивание окончаний для Правила 1.
# Недостаточно одного подхода
def sort_inclusion(x):
    x = list(set(x))
    for i in range(100):
        if sort_inclusion_one_round(x)==x:
            return sort_inclusion_one_round(x)
        x = sort_inclusion_one_round(x)    
    return ['Cannot sort affixes. Some affix can be repeated.']

affixes = sort_inclusion(affixes)

## Загрузка корпуса 

Корпус словоформ  wordforms.csv опубликован https://github.com/Sakha-Language-Processing/wordforms
и
https://www.kaggle.com/c/sakha-lemmatizer

In [69]:
# !wget https://github.com/Sakha-Language-Processing/wordforms/blob/main/wordforms202104.zip
# !unzip wordforms202104.zip
# !unzip /content/wordforms.csv.zip

--2021-04-18 03:48:13--  https://github.com/Sakha-Language-Processing/wordforms/blob/main/wordforms202104.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘wordforms202104.zip’

wordforms202104.zip     [ <=>                ] 106.73K  --.-KB/s    in 0.04s   

2021-04-18 03:48:13 (2.81 MB/s) - ‘wordforms202104.zip’ saved [109296]

Archive:  wordforms202104.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of wordforms202104.zip or
        wordforms202104.zip.zip, and cannot find wordforms202104.zip.ZIP, period.


## Выявление и стемминг

In [66]:
df = pd.read_csv('wordforms.csv')
print('Загружено ',len(df), 'словоформ')
print('Первоначальный текст состоял из ',df['occurence'].sum(), 'вхождений')

# Применяем правило 2
df['wordcanon'] = df['wordform'].apply(change_to_a)

# Применяем правило 1
df['verb'] = df['wordcanon'].apply(isitaverb)
df = df[df['verb']>0]
print('Выявлено ',len(df), 'словоформ глаголов')
print('В первоначальном тексте было не менее ',df['occurence'].sum(), 'вхождений глаголов')

# Стемминг
df['stem']=''
for i, row in df.iterrows():
    if df.at[i,'verb']!=0:   
         df.at[i,'stem'] = df.at[i,'wordform'][:df.at[i,'verb']] 

# Отбрасываем исключения
for item in exceptions:
    df = df[df['stem']!=item]

# Сохранение результатов
df = df.sort_values(by='stem')[['wordform','occurence','stem','wordcanon']]
df[['wordform','stem']].to_csv('verbs.csv',index=False) 

stemming = pd.DataFrame(list(set(list(df['stem']))))
stemming.columns = ['stem']
stemming.to_csv('verb_stems.csv',index=False)

Загружено  98568 словоформ
Первоначальный текст состоял из  1028838 вхождений
Выявлено  4640 словоформ глаголов
В первоначальном тексте было не менее  23294 вхождений глаголов
